<a href="https://colab.research.google.com/github/jihun0423/Pipeline-Config/blob/main/%EC%A0%84%EC%B2%98%EB%A6%AC%2C_ML%2C_DL_%EC%BD%94%EB%93%9C%EB%93%A4_%EB%AA%A8%EC%9D%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# encoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler


from sklearn.preprocessing import MinMaxScaler

# machine learning models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor

!pip install pycaret
from pycaret.regression import *

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

!pip install skorch

from skorch import NeuralNetRegressor
from sklearn.base import BaseEstimator, TransformerMixin

import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, KFold
from sklearn import set_config

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt

# ML & DL PIPELINE
어느정도 기본적인 전처리 (결측치 제거 & EDA까지 완료한 뒤에 인풋)

In [ ]:
def pipeline(cols, cat_cols, method='lr'):
  cat_transformer = OneHotEncoder(sparse_output=False,handle_unknown='ignore')

  num_cols = list(set(cols)-set(cat_cols))
  num_cols.sort()
  num_transformer = RobustScaler()

  preprocessor = ColumnTransformer(transformers=[("num",num_transformer,num_cols),
                                                 ("cat",cat_transformer,cat_cols)])


  class FloatTransformer(BaseEstimator, TransformerMixin):
      def __init__(self):
            pass
      def fit(self, X, y=None):
            return self
      def transform(self, x):
            return np.array(x, dtype=np.float32)

  if method == 'lr':
    ml = LinearRegression(fit_intercept=True)
  elif method == 'rf':
    ml = RandomForestRegressor()
  elif method == 'xgb':
    ml = XGBRegressor()
  elif method == 'etr':
    ml = ExtraTreesRegressor()
  elif method == 'torch':
    ml = NeuralNetRegressor(Regressor(),max_epochs=100,criterion=nn.MSELoss(),device='cuda:0',optimizer=optim.Adam,optimizer__lr = 0.01)

  model = Pipeline(steps=[("preprocessor",preprocessor),
                          ("float64to32",FloatTransformer()),
                          ("ml",ml)])

  return model

In [ ]:
class Regressor(nn.Module):                                                   # 기본적인 MLP
  def __init__(self):
    super().__init__()
    self.model = nn.Sequential(nn.Linear(62,30),
                               nn.ReLU(),
                               nn.Linear(30,30),
                               nn.ReLU(),
                               nn.Linear(30,10),
                               nn.Dropout(0.2),
                               nn.ReLU(),
                               nn.Linear(10,1),
                               )

  def forward(self,x):
    x=self.model(x)
    return x

In [ ]:
from sklearn import set_config
set_config(display='diagram')                                                        # pipeline 시각화 설정
model = pipeline(list(cols),list(cat_cols), method="torch")                          # pipeline 함수에 모든 컬럼 리스트, 카테고리(범주형)컬럼 리스트, 훈련 할 모델 입력
model    # pipeline 확인

In [ ]:
model = pipeline(list(cols),list(cat_cols), method="torch")                          # pipeline 함수에 모든 컬럼 리스트, 카테고리(범주형)컬럼 리스트, 훈련 할 모델 입력
model.fit(X_train, y_train.astype(np.float32).values.reshape(-1, 1))                 # method='torch'일 경우에만 np.float32로 변경, 아닐 경우는 y_train으로
model

In [ ]:
from sklearn import set_config
set_config(display='diagram')
pp = pipeline(list(cols),list(cat_cols),method='rf')
pp

# 이미지 데이터 불러오기 (파일 경로를 통해)

## 필요한 패키지들 임포트 및 기본 설정

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':32,
    'SEED':41
}

## 데이터 프레임에 이미지 경로, 라벨 저장

In [ ]:
base_dir = '/content/gdrive/MyDrive/open (2)/'
train_folder = glob.glob(base_dir + 'train/*')
all_img_list = glob.glob(base_dir + 'train/*/*')

df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list            # 이미지 경로 데이터프레임에 저장
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])         # Label명 추출

In [ ]:
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['label'])   # 라벨 인코딩

## 시각화

In [ ]:
plt.figure(figsize=(12,8))
sns.countplot(data=df, y='label',  palette='crest', dodge=False)
plt.show()                     # 라벨 별 분포 확인

In [ ]:
plt.figure(figsize = (15,12))
for idx, i in enumerate(train_df.label.unique()):
    plt.subplot(4, 7, idx+1)

    df = train_df[train_df['label'] == i].reset_index(drop = True)
    # image_path = df.loc[random.randint(0, len(df))-1, 'path']
    image_path = df.loc[random.randint(0, len(df)-1), 'path']
    img = Image.open(image_path)
    img = img.resize((224,224))
    plt.imshow(img)
    plt.axis('off')
    plt.title(i)
plt.tight_layout()
plt.show()                    # 이미지 일부 확인

## TrainTestSplit

In [ ]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## 커스텀 데이터셋 (이미지 경로 활용)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms

    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        image = cv2.imread(img_path)                 # PIL로 변환

        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

In [ ]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            ToTensorV2()
                            ])                                 # Albumentation패키지를 이용한 augmentation

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# 커스텀 데이터셋(이미 이미지가 로드 되어있을 경우)

In [ ]:
tf = transforms.ToTensor()

class CustomDataset(Dataset):
    def __init__(self, img, label, transforms=None):
        self.img = img
        self.label = label
        self.transforms = transforms

    def __getitem__(self, index):
        img = self.img[index]

        if self.transforms is not None:
            image = self.transforms(img)

        if self.label is not None:
            label = self.label[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img)

In [ ]:
augset=CustomDataset(img=aug_img,label=l.values,transforms=tf)

## 이미지들의 표준화를 위한 평균, 표준편차 구하기

In [ ]:
def print_stats(dataset):
    imgs = np.array([img.numpy() for img, _ in dataset])
    print(f'shape: {imgs.shape}')

    min_r = np.min(imgs, axis=(2, 3))[:, 0].min()
    min_g = np.min(imgs, axis=(2, 3))[:, 1].min()
    min_b = np.min(imgs, axis=(2, 3))[:, 2].min()

    max_r = np.max(imgs, axis=(2, 3))[:, 0].max()
    max_g = np.max(imgs, axis=(2, 3))[:, 1].max()
    max_b = np.max(imgs, axis=(2, 3))[:, 2].max()

    mean_r = np.mean(imgs, axis=(2, 3))[:, 0].mean()
    mean_g = np.mean(imgs, axis=(2, 3))[:, 1].mean()
    mean_b = np.mean(imgs, axis=(2, 3))[:, 2].mean()

    std_r = np.std(imgs, axis=(2, 3))[:, 0].std()
    std_g = np.std(imgs, axis=(2, 3))[:, 1].std()
    std_b = np.std(imgs, axis=(2, 3))[:, 2].std()

    print(f'min: {min_r, min_g, min_b}')
    print(f'max: {max_r, max_g, max_b}')
    print(f'mean: {mean_r, mean_g, mean_b}')
    print(f'std: {std_r, std_g, std_b}')

In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
    ])

In [ ]:
train_dataset = CustomDataset(df['img_path'].values, df['label'].values, transform)
print_stats(train_dataset)

In [ ]:
for i, (inputs,labels) in enumerate(augmentate_loader):
  aug_img = to_pil_image(inputs[0])
  aug_img.save('/content/gdrive/MyDrive/open (2)/aug/'+f'{int(labels)}/'+f'{i}.jpg')